In [ ]:
!pip install spacy stanza textstat pymorphy2 --quiet
!python -m spacy download ru_core_news_md --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.6/176.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 73.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [ ]:
!pip install spacy

In [ ]:
!pip install pymorphy3 --quiet


In [ ]:
!pip install language_tool_python --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import language_tool_python
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
import spacy
import stanza
import pymorphy2

In [ ]:
import textstat

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()


In [ ]:
stanza.download("ru")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [ ]:
nlp_spacy = spacy.load("ru_core_news_md")
nlp_stanza = stanza.Pipeline("ru", processors="tokenize,pos,lemma,depparse")


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
tool = language_tool_python.LanguageTool('ru')

In [ ]:
def get_error_metrics(text):
    """
    Robust error counting for Russian using LanguageTool.
    Filters out 'CASING' noise to focus on deep proficiency signals.
    """
    try:
        matches = tool.check(text)
    except:
        return 0.0, 0.0

    grammar_errors = 0
    spelling_errors = 0

    for match in matches:
        rule_id = getattr(match, 'ruleId', '').upper()
        category = getattr(match, 'category', '').upper()

        # --- 1. IGNORE NOISE ---
        # We skip casing errors because they often mask real grammar,
        # and checking for capitalization is not our primary goal.
        if 'CASING' in category or 'UPPERCASE_SENTENCE_START' in rule_id:
            continue

        # --- 2. SPELLING (Out of Vocabulary) ---
        # MORFOLOGIK rules mean the word was not found in the dict.
        # TYPOS means the tool thinks it's a typo.
        is_spelling = (
            'SPELLER' in rule_id or
            'MORFOLOGIK' in rule_id or
            'TYPOS' in category
        )

        # --- 3. GRAMMAR (Contextual) ---
        # Everything else (Agreement, Government, Redundancy) is Grammar.
        if is_spelling:
            spelling_errors += 1
        else:
            grammar_errors += 1

    token_count = len(text.split())
    if token_count == 0:
        return 0.0, 0.0

    spelling_rate = (spelling_errors / token_count) * 100
    grammar_rate = (grammar_errors / token_count) * 100

    return spelling_rate, grammar_rate

In [ ]:
tool.check('Я видю')

[Match({'rule_id': 'MORFOLOGIK_RULE_RU_RU', 'message': 'Возможно найдена орфографическая ошибка.', 'replacements': ['ведь', 'вид', 'веда', 'федю', 'иею', 'вею', 'вед', 'ведаю', 'ведд', 'веде', 'веди', 'ведою', 'ведя', 'веду', 'веды', 'велю', 'веню', 'верю', 'вида', 'видаю', 'виде', 'видя', 'виду', 'видь', 'виды', 'вилю', 'виню', 'витю', 'Федю', 'вид ю'], 'offset_in_context': 2, 'context': 'Я видю', 'offset': 2, 'error_length': 4, 'category': 'TYPOS', 'rule_issue_type': 'misspelling', 'sentence': 'Я видю'})]

In [ ]:
tool.check('Два столы.')

[Match({'rule_id': 'Sklonenije_NUM_NN', 'message': 'Числительное не согласуется с существительным по падежу: «Два стола».', 'replacements': ['Два стола'], 'offset_in_context': 0, 'context': 'Два столы.', 'offset': 0, 'error_length': 9, 'category': 'GRAMMAR', 'rule_issue_type': 'uncategorized', 'sentence': 'Два столы.'})]

In [ ]:
get_error_metrics('Я видю')

(50.0, 0.0)

In [ ]:
get_error_metrics('Два столы.')

(0.0, 50.0)

In [ ]:
def mattr(tokens, window=50):
    """Moving-Average Type Token Ratio."""
    if len(tokens) < window:
        return len(set(tokens)) / len(tokens)
    scores = []
    for i in range(len(tokens) - window + 1):
        window_tokens = tokens[i:i+window]
        scores.append(len(set(window_tokens)) / window)
    return np.mean(scores)

In [ ]:
def extract_features(text):
    # Run NLP Pipelines
    doc_spacy = nlp_spacy(text)
    doc_stanza = nlp_stanza(text)

    # Basic Stats
    sentences = list(doc_spacy.sents)
    tokens = [t for t in doc_spacy]
    words = [t.text for t in tokens if t.is_alpha] # Filter punctuation

    num_sentences = len(sentences) if len(sentences) > 0 else 1
    num_tokens = len(tokens) if len(tokens) > 0 else 1
    num_words = len(words)

    # Sentence / Token Lengths
    sent_lengths_chars = [len(s.text) for s in sentences]
    sent_lengths_tokens = [len([t for t in s if t.is_alpha]) for s in sentences]
    token_lengths_chars = [len(w) for w in words]

    def stats(arr):
        if len(arr) == 0: return (0,0,0,0)
        return (np.mean(arr), np.median(arr), np.min(arr), np.max(arr))

    mean_sc, med_sc, min_sc, max_sc = stats(sent_lengths_chars)
    mean_st, med_st, min_st, max_st = stats(sent_lengths_tokens)
    mean_tc, med_tc, min_tc, max_tc = stats(token_lengths_chars)

    # Lexical Metrics
    # lexical_diversity = len(set(words)) / len(words) if words else 0
    mattr_score = mattr(words, window=50)

    # Lexical Density (Content Words / Total Tokens)
    content_words = [t for t in doc_spacy if t.pos_ in ["NOUN", "VERB", "ADJ", "ADV"]]
    lexical_density = len(content_words) / num_words # not counting punctuation

    # POS Ratios
    pos_counts = doc_spacy.count_by(spacy.attrs.POS)

    noun_ratio = pos_counts.get(spacy.symbols.NOUN, 0) / num_tokens
    verb_ratio = pos_counts.get(spacy.symbols.VERB, 0) / num_tokens
    adj_ratio  = pos_counts.get(spacy.symbols.ADJ, 0)  / num_tokens
    adv_ratio  = pos_counts.get(spacy.symbols.ADV, 0)  / num_tokens
    pron_ratio = pos_counts.get(spacy.symbols.PRON, 0) / num_tokens

    # Function Word Ratio (Prepositions, conjunctions, particles)
    func_count = len([t for t in doc_spacy if t.pos_ in ["ADP", "AUX", "SCONJ", "CCONJ", "PART", "PRON", "DET"]])
    function_word_ratio = func_count / num_tokens

    # Readability
    ari = textstat.automated_readability_index(text)
    fres = textstat.flesch_reading_ease(text)
    smog = textstat.smog_index(text)
    cli = textstat.coleman_liau_index(text)

    # Syntactic (Clauses & Tree Depth)
    total_clauses = 0
    depths = []

    for sent in doc_stanza.sentences:
        # 1. Tree Depth
        def get_depth(node_id, nodes_dict):
            if node_id == 0: return 0
            head_id = nodes_dict[node_id].head
            if head_id == 0: return 1
            return 1 + get_depth(head_id, nodes_dict)

        # Map words for easier lookup
        nodes = {w.id: w for w in sent.words}
        sent_depths = [get_depth(w.id, nodes) for w in sent.words]
        depths.append(max(sent_depths) if sent_depths else 0)

        # 2. Clause Count (Finite Verbs as proxy for clauses)
        # Using 'VerbForm=Fin' feature from Stanza
        clauses_in_sent = 0
        for w in sent.words:
            if w.upos == "VERB":
                 # Check if it's a finite verb
                 if w.feats and "VerbForm=Fin" in w.feats:
                     clauses_in_sent += 1
                 # Fallback: if it's the root or a conjunct, count it
                 elif w.deprel in ["root", "conj", "ccomp", "advcl"]:
                     clauses_in_sent += 1
        total_clauses += clauses_in_sent

    avg_tree_depth = np.mean(depths) if depths else 0
    clauses_per_sentence = total_clauses / num_sentences

    # Errors (LanguageTool)
    sp_rate, gr_rate = get_error_metrics(text)

    # Final Dictionary
    return {
        # Volume
        # "num_sentences": num_sentences,
        # "num_tokens": num_tokens,
        # "num_words": num_words,

        # Sentence Stats
        "sent_len_tokens_mean": mean_st,
        "sent_len_tokens_max": max_st,

        # Token Stats
        "token_len_chars_mean": mean_tc,
        "token_len_chars_max": max_tc,

        # Lexical
        # "lexical_diversity": lexical_diversity,
        "mattr": mattr_score,
        "lexical_density": lexical_density,

        # POS Ratios
        "noun_ratio": noun_ratio,
        "verb_ratio": verb_ratio,
        "adj_ratio": adj_ratio,
        "pron_ratio": pron_ratio,
        "function_word_ratio": function_word_ratio,

        # Syntactic
        # "num_clauses": total_clauses, # Raw count
        "clauses_per_sentence": clauses_per_sentence, # Normalized
        "avg_tree_depth": avg_tree_depth,

        # Readability
        "ari": ari,
        "fres": fres,
        "smog": smog,
        "cli": cli,

        # Errors
        "sp_rate": sp_rate,
        "gr_rate": gr_rate
    }

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("training_data.csv")

In [ ]:
import numpy as np

In [ ]:
from tqdm.notebook import tqdm


In [ ]:
feature_rows = [extract_features(t) for t in tqdm(df["text"])]
features_df = pd.DataFrame(feature_rows)

features_df["label"] = df["labels"]
features_df.to_csv("training_features.csv", index=False)
features_df.head()

  0%|          | 0/1138 [00:00<?, ?it/s]

,sent_len_tokens_mean,sent_len_tokens_max,token_len_chars_mean,token_len_chars_max,mattr,lexical_density,noun_ratio,verb_ratio,adj_ratio,pron_ratio,function_word_ratio,clauses_per_sentence,avg_tree_depth,ari,fres,smog,cli,sp_rate,gr_rate,label
0,8.571429,17,5.391667,11,0.905070,0.600000,0.276596,0.078014,0.078014,0.035461,0.340426,0.642857,3.642857,9.507091,112.943846,3.1291,12.500840,10.084034,5.042017,5.0
1,3.000000,3,2.333333,4,1.000000,0.333333,0.000000,0.250000,0.000000,0.250000,0.500000,1.000000,2.000000,-7.370000,119.190000,3.1291,-12.133333,0.000000,0.000000,4.0
2,5.000000,5,3.400000,6,1.000000,0.400000,0.166667,0.166667,0.000000,0.166667,0.500000,2.000000,3.000000,-0.090000,117.160000,3.1291,-2.000000,20.000000,0.000000,2.0
3,9.400000,13,4.670213,10,0.876444,0.627660,0.198413,0.111111,0.063492,0.103175,0.301587,1.000000,3.909091,6.973226,112.897000,3.1291,8.658696,16.129032,2.150538,4.0
4,10.529412,27,5.296089,11,0.855846,0.625698,0.212389,0.141593,0.070796,0.048673,0.269912,1.470588,4.235294,10.024871,108.799262,3.1291,12.421469,4.494382,1.123596,5.0


In [ ]:
# normalization?

In [ ]:
list(features_df.columns.values)


['sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate',
 'label']

In [ ]:
import pandas as pd

df = pd.read_csv("training_features.csv")
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'training_features.csv'

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["label"])
y = df["label"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train_scaled, y_train)


LinearRegression()

current:


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score
import numpy as np

y_pred = reg.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

y_pred_rounded = np.clip(np.round(y_pred), y.min(), y.max()).astype(int)
qwk = cohen_kappa_score(y_test, y_pred_rounded, weights="quadratic")

print("RMSE:", rmse)
print("MAE:", mae)
print("QWK:", qwk)


RMSE: 1.030226286506738
MAE: 0.7766445250038132
QWK: 0.7220595729373795


with lengths

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score
import numpy as np

y_pred = reg.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

y_pred_rounded = np.clip(np.round(y_pred), y.min(), y.max()).astype(int)
qwk = cohen_kappa_score(y_test, y_pred_rounded, weights="quadratic")

print("RMSE:", rmse)
print("MAE:", mae)
print("QWK:", qwk)


RMSE: 1.0016775312014135
MAE: 0.7639388732306411
QWK: 0.7300683708402806


In [ ]:
feature_importance = pd.Series(reg.coef_, index=X.columns).sort_values()
feature_importance


,0
sp_rate,-0.391742
noun_ratio,-0.370589
ari,-0.342721
verb_ratio,-0.143869
pron_ratio,-0.141266
gr_rate,-0.128837
adj_ratio,-0.047997
smog,0.009366
sent_len_tokens_mean,0.070492
clauses_per_sentence,0.127982


In [ ]:
# 1. Extract
text = "Здраствуйте, пастафьте мне максимальную аценку"
feat = extract_features(text)

# 2. Convert to DF (Ensure order matches training!)
feature_order = [
'sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate'
]

feat_row = pd.DataFrame([feat], columns=feature_order)

# 3. Scale & Predict
feat_scaled = scaler.transform(feat_row)
pred_continuous = reg.predict(feat_scaled)[0]
pred_label = int(np.clip(round(pred_continuous), 0, 7))

print(f"Predicted Score: {pred_continuous:.2f}")
print(f"Rounded Label: {pred_label}")
print(f"Errors Found: Spelling={feat['sp_rate']:.1f}, Grammar={feat['gr_rate']:.1f}")

Predicted Score: 3.81
Rounded Label: 4
Errors Found: Spelling=60.0, Grammar=0.0


In [ ]:
# 1. Extract
text = "Здравствуйте, постафьте мне максимальную оценку"
feat = extract_features(text)

# 2. Convert to DF (Ensure order matches training!)
feature_order = [
'sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate'
]

feat_row = pd.DataFrame([feat], columns=feature_order)

# 3. Scale & Predict
feat_scaled = scaler.transform(feat_row)
pred_continuous = reg.predict(feat_scaled)[0]
pred_label = int(np.clip(round(pred_continuous), 0, 7))

print(f"Predicted Score: {pred_continuous:.2f}")
print(f"Rounded Label: {pred_label}")
print(f"Errors Found: Spelling={feat['sp_rate']:.1f}, Grammar={feat['gr_rate']:.1f}")

Predicted Score: 5.89
Rounded Label: 6
Errors Found: Spelling=20.0, Grammar=0.0


In [ ]:
# 1. Extract
text = "Здравствуйте!"
feat = extract_features(text)

# 2. Convert to DF (Ensure order matches training!)
feature_order = [
'sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate'
]

feat_row = pd.DataFrame([feat], columns=feature_order)

# 3. Scale & Predict
feat_scaled = scaler.transform(feat_row)
pred_continuous = reg.predict(feat_scaled)[0]
pred_label = int(np.clip(round(pred_continuous), 0, 7))

print(f"Predicted Score: {pred_continuous:.2f}")
print(f"Rounded Label: {pred_label}")
print(f"Errors Found: Spelling={feat['sp_rate']:.1f}, Grammar={feat['gr_rate']:.1f}")

Predicted Score: 7.65
Rounded Label: 7
Errors Found: Spelling=0.0, Grammar=0.0


In [ ]:
# 1. Extract
text = "Здраствуйте, пастафьте мне максимальную аценку. Здраствуйте, пастафьте мне максимальную аценку. Здраствуйте, пастафьте мне максимальную аценку. Здраствуйте, пастафьте мне максимальную аценку"
feat = extract_features(text)

# 2. Convert to DF (Ensure order matches training!)
feature_order = [
'sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate'
]

feat_row = pd.DataFrame([feat], columns=feature_order)

# 3. Scale & Predict
feat_scaled = scaler.transform(feat_row)
pred_continuous = reg.predict(feat_scaled)[0]
pred_label = int(np.clip(round(pred_continuous), 0, 7))

print(f"Predicted Score: {pred_continuous:.2f}")
print(f"Rounded Label: {pred_label}")
print(f"Errors Found: Spelling={feat['sp_rate']:.1f}, Grammar={feat['gr_rate']:.1f}")

Predicted Score: 1.68
Rounded Label: 2
Errors Found: Spelling=60.0, Grammar=0.0


In [ ]:
# 1. Extract
text = "a"
feat = extract_features(text)

# 2. Convert to DF (Ensure order matches training!)
feature_order = [
'sent_len_tokens_mean',
 'sent_len_tokens_max',
 'token_len_chars_mean',
 'token_len_chars_max',
 'mattr',
 'lexical_density',
 'noun_ratio',
 'verb_ratio',
 'adj_ratio',
 'pron_ratio',
 'function_word_ratio',
 'clauses_per_sentence',
 'avg_tree_depth',
 'ari',
 'fres',
 'smog',
 'cli',
 'sp_rate',
 'gr_rate'
]

feat_row = pd.DataFrame([feat], columns=feature_order)

# 3. Scale & Predict
feat_scaled = scaler.transform(feat_row)
pred_continuous = reg.predict(feat_scaled)[0]
pred_label = int(np.clip(round(pred_continuous), 0, 7))

print(f"Predicted Score: {pred_continuous:.2f}")
print(f"Rounded Label: {pred_label}")
print(f"Errors Found: Spelling={feat['sp_rate']:.1f}, Grammar={feat['gr_rate']:.1f}")

Predicted Score: -13.49
Rounded Label: 0
Errors Found: Spelling=100.0, Grammar=0.0
